In [2]:
#gzoltar_files_path = "/Users/lorenapacheco/Concordia/Masters/BugReportsMining/gzoltar_files"
gzoltar_files_path = r"C:\Users\loren\Concordia\Masters\BugReportsMining\gzoltar_files"
#out = '/Users/lorenapacheco/Concordia/Masters/BugReportsMining/ochiaiScores2.0/Ochiai1/'
out = r"C:\Users\loren\Concordia\Masters\BugReportsMining\ochiaiScores2.0\Ochiai1/"
#data_file_path = "/Users/lorenapacheco/Concordia/Masters/BugReportsMining/data/merged_data_production_bug_reports.json"
data_file_path = r"C:\Users\loren\Concordia\Masters\BugReportsMining\data\merged_data_production_bug_reports.json"

## Reading the coverage data

In [3]:
import csv
import re
import os
import json


def json_file_to_dict(file):
    data = {}
    with open(os.path.join(file), 'r') as fp:
        data = json.load(fp)
    fp.close()
    return data

def read_matrix_file(file_path):
    statements_covered_per_test = []
    test_passed = []

    with open(os.path.join(file_path, "matrix.txt"), 'r') as f:
        for line in f:
            row = [int(num) for num in line.strip()[:-1].split()] # The last number is the test execution result (pass or fault) and needs to be removed from the coverage obj
            # sign = line.strip()[-1]
            statements_covered_per_test.append(row)
    return statements_covered_per_test

def read_spectra_file(file_path):
    lines_of_code_obj_list = []
    pattern = r'^(.*?)#(.*?)\((.*?)\):(\d+)$'
    with open(os.path.join(file_path, "spectra.csv"), 'r') as file:
        first_line = True
        for line in file:
            # Skip the first line
            if first_line:
                first_line = False
                continue
            composed_str = line
            match = re.search(pattern, composed_str)
            if match is None:
                print("match not found")
                print(composed_str)
                continue
            class_name = match.group(1)
            method_name = match.group(2)
            method_parameters = match.group(3)
            line_number = int(match.group(4))
            lines_of_code_obj_list.append({
                "class_name": class_name,
                "method_name": method_name,
                "method_parameters": method_parameters,
                "line_number": line_number,
            })
    return lines_of_code_obj_list

def read_tests_file(file_path):
    test_names = []
    test_results = []
    first_row = True
    with open(os.path.join(file_path, "tests.csv"), 'r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            if first_row:
                first_row = False
                continue
            test_name = row[0]
            test_result = False
            if row[1] == "PASS":
                test_result = True
            test_names.append(test_name)
            test_results.append(test_result)
    return test_names, test_results


bugs_data = json_file_to_dict(data_file_path)
coverage_data = {}
for project in bugs_data.keys():
    project_gzoltar_folder = os.path.join(gzoltar_files_path, project)
    if not os.path.exists(project_gzoltar_folder):
        print("Gzoltar folder not fount for the project "+ project)
        print("Skipping!!!!! ")
        continue
    if not project in coverage_data.keys():
        coverage_data[project] = {}
    for bug_id in bugs_data[project].keys():
        bug_gzoltar_folder = os.path.join(project_gzoltar_folder, bug_id)
        if not os.path.exists(bug_gzoltar_folder):
            print("Gzoltar folder not fount for the bugId "+ project+ "-" + bug_id)
            print("Skipping!!!!! ")
            continue
        coverage_data[project][bug_id] = {}
        coverage_data[project][bug_id]["statements_covered_per_test"] = read_matrix_file(bug_gzoltar_folder)
        print("Number of tests in bug "+ project+ "-" + bug_id + " - " + str(len(coverage_data[project][bug_id]["statements_covered_per_test"])))
        coverage_data[project][bug_id]["lines_of_code_obj_list"] = read_spectra_file(bug_gzoltar_folder)
        test_names, test_results = read_tests_file(bug_gzoltar_folder)
        coverage_data[project][bug_id]["test_names"] = test_names
        coverage_data[project][bug_id]["test_results"] = test_results
print("Done")

Number of tests in bug Cli-14 - 544
Number of tests in bug Cli-5 - 102
Number of tests in bug Closure-106 - 2595
Number of tests in bug Closure-125 - 8158
Number of tests in bug Closure-143 - 0
Number of tests in bug Closure-152 - 0
Number of tests in bug Closure-2 - 7828
Number of tests in bug Closure-27 - 7482
Number of tests in bug Closure-34 - 7414
Number of tests in bug Closure-37 - 7373
Number of tests in bug Closure-55 - 7168
Number of tests in bug Codec-8 - 304
Number of tests in bug Collections-28 - 5284
Number of tests in bug Compress-1 - 70
Number of tests in bug Compress-12 - 316
Number of tests in bug Compress-14 - 320
Number of tests in bug Compress-17 - 351
Number of tests in bug Compress-18 - 357
Number of tests in bug Compress-23 - 467
Number of tests in bug Compress-32 - 614
Number of tests in bug Csv-12 - 198
Number of tests in bug Csv-4 - 178
Number of tests in bug Gson-12 - 1017
Number of tests in bug Gson-8 - 992
Number of tests in bug JacksonCore-4 - 239
Number o

## Removing the failing test results

In [4]:
for project in coverage_data.keys():
    print(project)
    for bug_id in coverage_data[project].keys():
        print(bug_id)
        failing_tests_indexes = []
        print(len(coverage_data[project][bug_id]["test_names"]))
        for index, test in enumerate(coverage_data[project][bug_id]["test_names"]):
            if not coverage_data[project][bug_id]["test_results"][index]: # failing test
                failing_tests_indexes.append(index)
        print( "----")
        print (failing_tests_indexes)
        print(str(len(failing_tests_indexes)) + " tests to be deleted")
        for index in reversed(failing_tests_indexes):
            del coverage_data[project][bug_id]["test_names"][index]
            del coverage_data[project][bug_id]["test_results"][index]
            del coverage_data[project][bug_id]["statements_covered_per_test"][index]
        print (len(coverage_data[project][bug_id]["test_names"]))

print("Done")

Cli
14
544
----
[196]
1 tests to be deleted
543
5
102
----
[0, 17, 57]
3 tests to be deleted
99
Closure
106
2595
----
[547, 548, 646, 1400]
4 tests to be deleted
2591
125
8158
----
[674]
1 tests to be deleted
8157
143
0
----
[]
0 tests to be deleted
0
152
0
----
[]
0 tests to be deleted
0
2
7828
----
[1239]
1 tests to be deleted
7827
27
7482
----
[7243, 7244, 7245]
3 tests to be deleted
7479
34
7414
----
[4087]
1 tests to be deleted
7413
37
7373
----
[6227]
1 tests to be deleted
7372
55
7168
----
[4625]
1 tests to be deleted
7167
Codec
8
304
----
[292]
1 tests to be deleted
303
Collections
28
5284
----
[1018]
1 tests to be deleted
5283
Compress
1
70
----
[16]
1 tests to be deleted
69
12
316
----
[46]
1 tests to be deleted
315
14
320
----
[251]
1 tests to be deleted
319
17
351
----
[262]
1 tests to be deleted
350
18
357
----
[255]
1 tests to be deleted
356
23
467
----
[92, 190]
2 tests to be deleted
465
32
614
----
[273, 453]
2 tests to be deleted
612
Csv
12
198
----
[137]
1 tests to be

In [24]:
import os
import json
import math
import numpy as np


def dict_to_json_file(file, dic, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    with open(os.path.join(folder, file+'.json'), 'w') as fp:
        json.dump(dic, fp, sort_keys=True, indent=4)
    fp.close()

def get_unique_tests_that_cover_the_methods (stackTraceMethodsDetails):
    first_st_file = list(stackTraceMethodsDetails.keys())[0]
    first_st_method = list(stackTraceMethodsDetails[first_st_file].keys())[0]
    repeated_test_methods = list(stackTraceMethodsDetails[first_st_file][first_st_method]["tests_covering_the_method"])
    for st_file in stackTraceMethodsDetails.keys():
        for st_method in stackTraceMethodsDetails[st_file].keys():
            removal_list = []
            for test in repeated_test_methods:
                if test not in stackTraceMethodsDetails[st_file][st_method]["tests_covering_the_method"]:
                    removal_list.append(test)
            for test in removal_list:
                repeated_test_methods.remove(test)

    result = {}
    for st_file in stackTraceMethodsDetails.keys():
        for st_method in stackTraceMethodsDetails[st_file].keys():
            unique_test_methods = list(stackTraceMethodsDetails[st_file][st_method]["tests_covering_the_method"])
            removal_list = []
            for test in unique_test_methods:
                if test in repeated_test_methods:
                    removal_list.append(test)
            for test in removal_list:
                unique_test_methods.remove(test)
            if st_file not in result.keys():
                result[st_file]={}
            result[st_file][st_method] = unique_test_methods
    return result

def write_matrix_to_file(matrix, filename):
    with open(filename, 'w') as f:
        for row in matrix:
            f.write(' '.join(str(cell) for cell in row) + '\n')

def write_strings_list_to_csv(strings, filename):
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["name"])
        for string in strings:
            writer.writerow([string])

def write_two_lists_to_csv(list1, list2, filename):
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        for item1, item2 in zip(list1, list2):
            writer.writerow([item1, item2])

def store_methods_coverage_in_file(coverage, project, bug_id):
    project_gzoltar_folder = os.path.join(gzoltar_files_path, project)
    bug_gzoltar_folder =  os.path.join(project_gzoltar_folder , bug_id)

    methods_coverage_matrix = coverage["methods_covered_per_test"]
    methods_coverage_matrix_file_name =  os.path.join(bug_gzoltar_folder, "methods_matrix.txt")
    write_matrix_to_file(methods_coverage_matrix, methods_coverage_matrix_file_name)

    methods_spectra = coverage["methods_obj_list"]
    methods_coverage_matrix_file_name =  os.path.join(bug_gzoltar_folder, "methods_spectra.csv")
    write_strings_list_to_csv(methods_spectra, methods_coverage_matrix_file_name)

    test_names = coverage["test_names"]
    fake_test_results_ochiai_1 = coverage["fake_test_results"]
    fake_test_results_ochiai_1_file_name =  os.path.join(bug_gzoltar_folder, "fake_test_results_ochiai_1.csv")
    write_two_lists_to_csv(test_names, fake_test_results_ochiai_1, fake_test_results_ochiai_1_file_name)

bugs_data = json_file_to_dict(data_file_path)
for project in coverage_data.keys():
    for bug_id in coverage_data[project].keys():

        bug_data = bugs_data[project][bug_id]
        coverage = coverage_data[project][bug_id]

        print(project + " ---- " + bug_id)

        buggy_commit = bug_data["buggy_commit"]
        fake_failed_tests = []
        methods_list = []

        if bug_data["stackTraceMethodsDetails"] =={ }:
            print("The bug does not contain stackTraceMethodsDetails. Skipping.")
            continue

        print("* Part 1 - defining the fake failing tests")
        unique_test_methods = get_unique_tests_that_cover_the_methods(bug_data["stackTraceMethodsDetails"])
        for st_file in bug_data["stackTraceMethodsDetails"].keys():
            for st_method in bug_data["stackTraceMethodsDetails"][st_file].keys():
                if st_file in unique_test_methods.keys() and st_method in unique_test_methods[st_file].keys() and unique_test_methods[st_file] [st_method]!= []:
                    fake_failed_tests += unique_test_methods[st_file] [st_method]
        if not fake_failed_tests:
            for st_file in bug_data["stackTraceMethodsDetails"].keys():
                for st_method in bug_data["stackTraceMethodsDetails"][st_file].keys():
                    if "tests_covering_the_method" in bug_data["stackTraceMethodsDetails"][st_file][st_method].keys():
                        fake_failed_tests += bug_data["stackTraceMethodsDetails"][st_file][st_method]["tests_covering_the_method"]
        fake_failed_tests = list(set(fake_failed_tests))

        print("* Part 2 - preparing the fake_tests_status")
        coverage["fake_test_results"] = coverage["test_results"]
        for index, test in enumerate(coverage["test_names"]):
            if coverage["test_names"][index] in fake_failed_tests:
                coverage["fake_test_results"][index] = False

        print("* Part 3 - Modifying the Gzoltar results to be per method instead of statement")
        coverage["methods_covered_per_test"] = []
        coverage["methods_obj_list"] = []
        method_to_pos = {}

        #Getting the methods list
        for index_t, test_coverage in enumerate(coverage["statements_covered_per_test"]):
            for index_s, statement_instance in enumerate(coverage["statements_covered_per_test"][index_t]):
                lines_of_code_obj_list = coverage["lines_of_code_obj_list"][index_s]
                method_id =  lines_of_code_obj_list["class_name"].replace("$", ".") + "#" + lines_of_code_obj_list["method_name"]
                if method_id not in method_to_pos:
                    method_to_pos[method_id] = len(coverage["methods_obj_list"])
                    coverage["methods_obj_list"].append(method_id)


        # Getting the methods coverage matrix
        for index_t, test_coverage in enumerate(coverage["statements_covered_per_test"]):
            test_name = coverage["test_names"][index_t]
            # Pre=populating the matrix
            coverage["methods_covered_per_test"].append([])
            for method_name in coverage["methods_obj_list"]:
                coverage["methods_covered_per_test"][index_t].append("0")
            # Getting the coverage
            for index_s, statement_instance in enumerate(coverage["statements_covered_per_test"][index_t]):
                lines_of_code_obj_list = coverage["lines_of_code_obj_list"][index_s]
                method_id =  lines_of_code_obj_list["class_name"].replace("$", ".") + "#" + lines_of_code_obj_list["method_name"]
                position = method_to_pos[method_id]
                if coverage["methods_covered_per_test"][index_t][position]!= "1":
                    coverage["methods_covered_per_test"][index_t][position] = statement_instance
        print("Storing the gzoltar results extended per method in a file for future uses")
        store_methods_coverage_in_file(coverage, project, bug_id)
        print("store_methods_coverage_in_file operation completed")

        print("* Part 4 - Executing Ochiai")
        methods_ochiai_scores = {}
        for index_m, method_name in enumerate(coverage["methods_obj_list"]):
            n00 = 0
            n01 = 0
            n10 = 0
            n11 = 0
            s_o = 0
            for index_t, test_name in enumerate(coverage["test_names"]):
                if str(coverage["methods_covered_per_test"][index_t][index_m]) == "1":
                    if not coverage["fake_test_results"][index_t]:
                        n11 += 1
                    else:
                        n10 += 1
                else:
                    if not coverage["fake_test_results"][index_t]:
                        n01 += 1
                    else:
                        n00 += 1
            try:
                s_o = n11/math.sqrt((n11+n01)*(n11+n10))
            except ZeroDivisionError:
                s_o = 0
            methods_ochiai_scores[method_name] = s_o

        print("Number of fake passing tests: " + str(len(coverage["fake_test_results"])-len(fake_failed_tests)))
        print("Number of fake failing tests: " + str(len(fake_failed_tests)) + "\n")
        dict_to_json_file(bug_id, methods_ochiai_scores, out + project + "/")

print("Execution completed")

Cli ---- 14
* Part 1 - defining the fake failing tests
* Part 2 - preparing the fake_tests_status
* Part 3 - Modifying the Gzoltar results to be per method instead of statement
Storing the gzoltar results extended per method in a file for future uses
store_methods_coverage_in_file operation completed
* Part 4 - Executing Ochiai
Number of fake passing tests: 468
Number of fake failing tests: 75

387
387
Cli ---- 5
* Part 1 - defining the fake failing tests
* Part 2 - preparing the fake_tests_status
* Part 3 - Modifying the Gzoltar results to be per method instead of statement
Storing the gzoltar results extended per method in a file for future uses
store_methods_coverage_in_file operation completed
* Part 4 - Executing Ochiai
Number of fake passing tests: 78
Number of fake failing tests: 21

155
155
Closure ---- 106
The bug does not contain stackTraceMethodsDetails. Skipping.
Closure ---- 125
* Part 1 - defining the fake failing tests
* Part 2 - preparing the fake_tests_status
* Part 3 

Traceback (most recent call last):
  File "C:\Users\loren\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\loren\AppData\Local\Temp\ipykernel_14856\2887111744.py", line -1, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\loren\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\loren\anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "C:\Users\loren\anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1012, in structured_traceback
    return VerboseTB.structured_traceback(
  File "C:\Users\loren\anaconda3\lib\site-packages\IPython\core\ultratb.py", line 865, in structured_traceb